In [2]:
import pandas as pd
import numpy as np
import sklearn

### Methods:

1. Convert the 'cdc_case_earliest_dt' into a year_month feature. 
2. Train a 

In [3]:
df =  pd.read_csv("cleaned_surveillance_data.csv", index_col=0, parse_dates =True)


In [4]:
# add year_month
df.head()

,cdc_case_earliest_dt,cdc_report_dt,pos_spec_dt,onset_dt,current_status,sex,age_group,race_ethnicity_combined,hosp_yn,icu_yn,death_yn,medcond_yn
4,2020/11/20,2020/11/21,2020/11/19,2020/11/20,Laboratory-confirmed case,Male,10 - 19 Years,"Black, Non-Hispanic",No,No,No,Yes
8,2021/06/03,2021/06/06,2021/06/05,2021/06/03,Laboratory-confirmed case,Male,10 - 19 Years,"Black, Non-Hispanic",No,No,No,No
10,2020/10/29,2020/10/31,2020/10/29,2020/10/29,Laboratory-confirmed case,Male,10 - 19 Years,"Black, Non-Hispanic",No,No,No,No
17,2020/11/20,2020/11/21,2020/11/19,2020/11/20,Laboratory-confirmed case,Male,10 - 19 Years,"Black, Non-Hispanic",No,No,No,Yes
18,2021/01/09,2021/03/01,2021/01/15,2021/01/09,Laboratory-confirmed case,Male,10 - 19 Years,"Black, Non-Hispanic",No,No,No,No


In [5]:
df['month_year'] = pd.to_datetime(df['cdc_case_earliest_dt ']).dt.to_period('M')
df

,cdc_case_earliest_dt,cdc_report_dt,pos_spec_dt,onset_dt,current_status,sex,age_group,race_ethnicity_combined,hosp_yn,icu_yn,death_yn,medcond_yn,month_year
4,2020/11/20,2020/11/21,2020/11/19,2020/11/20,Laboratory-confirmed case,Male,10 - 19 Years,"Black, Non-Hispanic",No,No,No,Yes,2020-11
8,2021/06/03,2021/06/06,2021/06/05,2021/06/03,Laboratory-confirmed case,Male,10 - 19 Years,"Black, Non-Hispanic",No,No,No,No,2021-06
10,2020/10/29,2020/10/31,2020/10/29,2020/10/29,Laboratory-confirmed case,Male,10 - 19 Years,"Black, Non-Hispanic",No,No,No,No,2020-10
17,2020/11/20,2020/11/21,2020/11/19,2020/11/20,Laboratory-confirmed case,Male,10 - 19 Years,"Black, Non-Hispanic",No,No,No,Yes,2020-11
18,2021/01/09,2021/03/01,2021/01/15,2021/01/09,Laboratory-confirmed case,Male,10 - 19 Years,"Black, Non-Hispanic",No,No,No,No,2021-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
26887788,2020/05/03,2020/05/15,2020/05/06,2020/05/03,Laboratory-confirmed case,Female,50 - 59 Years,"White, Non-Hispanic",Yes,No,No,Yes,2020-05
26887792,2020/07/11,2020/09/24,2020/07/07,2020/09/06,Laboratory-confirmed case,Female,50 - 59 Years,"White, Non-Hispanic",No,No,No,Yes,2020-07
26887793,2020/07/19,2020/09/24,2020/07/17,2020/09/04,Laboratory-confirmed case,Female,50 - 59 Years,"White, Non-Hispanic",No,No,No,No,2020-07
26887798,2020/09/28,2020/10/13,NaN,NaN,Laboratory-confirmed case,Female,50 - 59 Years,"White, Non-Hispanic",No,No,No,No,2020-09


In [6]:
# make a month 

june_2020 = df[df['month_year'] == '2020-06']

june_2020.shape

(114774, 13)

In [7]:
# token validation. 
valid_month_year = set(df['month_year'].unique().strftime('%Y-%m'))
valid_sex = set(['Male', 'Female'])
valid_sex_age_group = set(df['age_group'].unique())
valid_race_ethnicity_combined = set(df['race_ethnicity_combined'].unique())
valid_medcond_yn = set(df['medcond_yn'].unique())


valid_sex_age_group

{'0 - 9 Years',
 '10 - 19 Years',
 '20 - 29 Years',
 '30 - 39 Years',
 '40 - 49 Years',
 '50 - 59 Years',
 '60 - 69 Years',
 '70 - 79 Years',
 '80+ Years',
 'Missing'}

In [10]:
# assume all inputs are valid

# def params_are_valid(month_year=None, sex=None, age_group=None, race_ethnicity_combined=None, medcond_yn=None):
#     # the most hacky validation I have ever written. 
#     if month_year is not None and month_year not in valid_month_year:
#         return False
#     elif sex is not None and sex not in valid_sex:
#         return False
#     elif age_group is not None and age_group not in valid_age_group:
#         return False
#     elif race_ethnicity_combined is not None and race_ethnicity_combined not in valid_race_ethnicity_combined:
#         return False
#     elif medcond_yn is not None and medcond_yn not in valid_medcond_yn:
#         return False
#     else:
#         return True


def get_death_value_counts(month_year=None, sex=None, age_group=None, race_ethnicity_combined=None, medcond_yn=None):
    """
            Returns a value count of deaths based on the params passed.
    """
    # if not params_are_valid(month_year,sex,age_group,race_ethnicity_combined,medcond_yn):
    #     raise ValueError
    
    if month_year is None:
        sub_set_condition = df['month_year'].notna() # this is just much faster than making an array of True vis np.Ones 
    else:
        sub_set_condition = df['month_year'] == month_year

    if sex is None:
        pass
    else:
        sub_set_condition = sub_set_condition & (df['sex'] == sex)

    if age_group is None:
        pass
    else:
        sub_set_condition = sub_set_condition & (df['age_group'] == age_group)

    if race_ethnicity_combined is None:
        pass
    else:
        sub_set_condition = sub_set_condition & (df['race_ethnicity_combined'] == race_ethnicity_combined)

    if medcond_yn is None:
        pass
    else:
        sub_set_condition = sub_set_condition & (df['medcond_yn'] == medcond_yn)

    print(type(sub_set_condition)) 

    sub_set_df = df[sub_set_condition]['death_yn'].value_counts()

    return sub_set_df


a = get_death_value_counts('2020-10', 'Male','80+ Years')


<class 'pandas.core.series.Series'>


In [11]:
a.to_dict()

{'No': 2851, 'Yes': 1245}